In [3]:
import numpy as np
import scipy as sp

import scipy
from scipy.stats import multivariate_normal
from scipy       import optimize

import skimage

import matplotlib.pyplot as plt

import pylab

from glob import glob
import sys
import os

from tqdm import tqdm

import math

from glob import glob

import matplotlib.pyplot as plt

import itk
from itk import TubeTK as ttk

import av
import ffmpeg

import json

import site
site.addsitedir('../../ARGUS')
from ARGUSUtils_IO import *
from ARGUSUtils_Linearization import *

In [49]:
def extrude_column(arr, col, val):
    col = int(col)
    first = 0
    for z in range(arr.shape[0]):
        pos = np.argmax(arr[z,:,col]==val)
        if pos>0:
            first = pos
            break
    for z in range(arr.shape[0]):
        pos = np.argmax(arr[z,:,col]==val)
        if pos>0:
            first = pos
        arr[z,:first,col]=0
        arr[z,first:,col]=val
    return arr
    
def extrude_annotations(pathname):
    data_paths = sorted(glob(pathname+"-Annotations-Linear/*_?????.mha"))
    data_overlay_paths = sorted(glob(pathname+"-Annotations-Linear/*_?????.interpolated-overlay.mha"))
    
    num_overlays = len(data_overlay_paths)
    for i in range(num_overlays):
        itkimg = itk.imread(data_overlay_paths[i], itk.UC)
        print("Processing", data_overlay_paths[i])
        
        # Eliminate rib annotation
        ovrarr = itk.GetArrayFromImage(itkimg)
        ovrarr = np.where(ovrarr==2,0,ovrarr)
        
        nos = 0
        rib = 0
        sli = 0
        for c in range(ovrarr.shape[2]):
            nos = np.count_nonzero(ovrarr[:,:,c]==1)
            rib = np.count_nonzero(ovrarr[:,:,c]==2)
            sli = np.count_nonzero(ovrarr[:,:,c]==3)
            if nos>rib and sli==0:
                ovrarr = extrude_column(ovrarr,c,1)
            #elif nos==0 and rib>0 and sli==0:
                #extrude_column(ovrimg,c,2)  #cause class 2 (rib) to be erased
            elif nos==0 and sli>rib:
                ovrarr = extrude_column(ovrarr,c,3)
            else:
                ovrarr[:,:,c]=0
                
        img = itk.imread(data_paths[i], itk.F)
        imMath = ttk.ImageMath.New(img)
        imMath.Blur(5)
        imMath.Threshold(0,50,0,1)
        imMath.Dilate(10,1,0)
        img = imMath.GetOutputUChar()
        connComp = ttk.SegmentConnectedComponents.New(img)
        connComp.SetMinimumVolume(1000000)
        connComp.Update()
        arr_mask = itk.GetArrayFromImage(connComp.GetOutput())
        arr_mask = np.where(arr_mask>0,1,0)
        keep = arr_mask * ovrarr
        #keep = np.where(keep==0,2,keep)
        for val in [1,3]:
            ovrarr = np.where(ovrarr==val,keep,ovrarr)
        (filepath,filename) = os.path.split(data_overlay_paths[i])
        new_name = filepath+"/"+filename.split('.')[0]+'.extruded-overlay.mha'
        print("Writing", new_name)
        itkimg = itk.GetImageFromArray(ovrarr.astype(np.ubyte))
        itk.imwrite(itkimg, new_name, compression=True)

In [51]:
#pathname = "../../Data/VFoldData/BAMC-PTXNoSliding"
pathname = "../../Data/VFoldData/BAMC-PTXSliding"
extrude_annotations(pathname)

Processing ../../Data/VFoldData/BAMC-PTXSliding-Annotations-Linear/004s_iimage_73815992352100_clean.interpolated-overlay.mha
Writing ../../Data/VFoldData/BAMC-PTXSliding-Annotations-Linear/004s_iimage_73815992352100_clean.extruded-overlay.mha
Processing ../../Data/VFoldData/BAMC-PTXSliding-Annotations-Linear/004s_iimage_74132233134844_clean.interpolated-overlay.mha
Writing ../../Data/VFoldData/BAMC-PTXSliding-Annotations-Linear/004s_iimage_74132233134844_clean.extruded-overlay.mha
Processing ../../Data/VFoldData/BAMC-PTXSliding-Annotations-Linear/019s_iimage_10705997566592_CLEAN.interpolated-overlay.mha
Writing ../../Data/VFoldData/BAMC-PTXSliding-Annotations-Linear/019s_iimage_10705997566592_CLEAN.extruded-overlay.mha
Processing ../../Data/VFoldData/BAMC-PTXSliding-Annotations-Linear/019s_iimage_10891015221417_clean.interpolated-overlay.mha
Writing ../../Data/VFoldData/BAMC-PTXSliding-Annotations-Linear/019s_iimage_10891015221417_clean.extruded-overlay.mha
Processing ../../Data/VFoldD

In [ ]:
pathname = "../../Data/TrainingData/AR-UNet/BAMC-PTXSliding"
extrude_annotations(pathname)

In [73]:
pathname = "../../Data/TestingData/BAMC-PTXNoSliding"
extrude_annotations(pathname)
pathname = "../../Data/TestingData/BAMC-PTXSliding"
extrude_annotations(pathname)

Processing ../../Data/TestingData/BAMC-PTXNoSliding-Annotations-Linear\image_417221672548_CLEAN.interpolated-overlay.mha
Writing ../../Data/TestingData/BAMC-PTXNoSliding-Annotations-Linear/image_417221672548_CLEAN.extruded-overlay.mha
Processing ../../Data/TestingData/BAMC-PTXNoSliding-Annotations-Linear\image_426794579576_CLEAN.interpolated-overlay.mha
Writing ../../Data/TestingData/BAMC-PTXNoSliding-Annotations-Linear/image_426794579576_CLEAN.extruded-overlay.mha
Processing ../../Data/TestingData/BAMC-PTXSliding-Annotations-Linear\image_3925135436261_clean.interpolated-overlay.mha
Writing ../../Data/TestingData/BAMC-PTXSliding-Annotations-Linear/image_3925135436261_clean.extruded-overlay.mha
Processing ../../Data/TestingData/BAMC-PTXSliding-Annotations-Linear\image_3929217595322_clean.interpolated-overlay.mha
Writing ../../Data/TestingData/BAMC-PTXSliding-Annotations-Linear/image_3929217595322_clean.extruded-overlay.mha
